# Explore MongoDB

In [2]:
import pymongo
from pymongo import MongoClient
from bson.son import SON

In [3]:
client = MongoClient()
db = client.lobbyradar

Entities = db.entities
Relations = db.relations

person_filter = { 'type' : 'person' }
entity_filter = { 'type' : 'entity' }
persons = Entities.find(person_filter)
entities = Entities.find(entity_filter)

__Entity typen__
werden alle zu einem `rdf:Class`

In [4]:
Entities.distinct('type')

[u'Bundesdatenschutzbeauftragte',
 u'Hausausweise',
 u'Mitglied',
 u'Position',
 u'Tochterfirma',
 u'Vorsitzender',
 u'activity',
 u'association',
 u'business',
 u'committee',
 u'consulting',
 u'donation',
 u'ececutive',
 u'entity',
 u'executive',
 u'government',
 u'lobbyist',
 u'member',
 u'mitglied',
 u'person',
 u'position',
 u'publication',
 u'sponsoring',
 u'subisdiary',
 u'subsidiary']

__Ralationship typen__
werden alle zu einem `rdf:Property`

In [5]:
Relations.distinct('type')
for row in Relations.distinct('type'):
    print row

Bundesdatenschutzbeauftragte
Hausausweise
Mitglied
Position
Tochterfirma
Vorsitzender
activity
association
business
committee
consulting
donation
ececutive
executive
government
lobbyist
member
mitglied
position
publication
sponsoring
subisdiary
subsidiary


__ Importer __

In [6]:
pipeline = [ {"$group": { "_id": "$importer", "count":{"$sum": 1}}}, 
             {"$sort": {"count":-1}} ]
for entity in Entities.aggregate(pipeline):
    print "%s (%i)" % (entity['_id'], entity['count'])

lobbyliste (32668)
bundestag (7398)
parteispenden (6372)
pr (4001)
None (2132)
dax (1213)
laender (905)
parteispenden13 (670)
db.clean (629)
seitenwechsler (569)
kabinette (518)
thinktanks (485)
parteispenden14 (15)
parteien (1)


# Organisationen
__Beispiel Organisation__

In [7]:
entities = Entities.find({'type' : 'entity'})
entities.next()

{u'_id': ObjectId('54d50d2fd2257f9a581e2c32'),
 u'aliases': [],
 u'created': datetime.datetime(2015, 2, 6, 18, 51, 27, 527000),
 u'data': [],
 u'importer': None,
 u'name': u'SPD Vorpommern',
 u'search': [u'spd vorpommern'],
 u'slug': u'spd vorpommern',
 u'tags': [u'spd'],
 u'type': u'entity',
 u'updated': datetime.datetime(2015, 2, 19, 17, 43, 54, 49000)}

__ Datentypen für Organisationen __

- source/Quelle => dc:source
- address/Adresse => vcard:ADR
- description/Beschreibung => dc:description
* members/Anzahl der Mitglieder (2118)
* organisations/Anzahl der Mitgliedsorganisationen (953)
- topic/Thema => foaf:topic
* url/URL (199)
* finance/Finanzierung (151)
* staff/Anzahl der Mitarbeiter (126)
* legalform/Rechtsform (28)
- link/Link => vcard:RELATED
* partei/Partei => org:Organization => own:Party
* begin/Von (5) => dc:data => own:from
* end/Bis (4) => dc:date => own:to
- www/Webseite => foaf:homepage

In [8]:
pipeline = [ {"$match" : { "type" : "entity" }}, 
             {"$unwind": "$data"}, 
             {"$group": { "_id": { "$concat": [ "$data.key", "/", "$data.desc"]}, "count":{"$sum": 1}}}, 
             {"$sort": {"count":-1}} ]
for entity in Entities.aggregate(pipeline):
    print "%s (%i)" % (entity['_id'], entity['count'])

source/Quelle (7495)
address/Adresse (6547)
description/Beschreibungstext (2370)
members/Anzahl der Mitglieder (2118)
organisations/Anzahl der Mitgliedsorganisationen (953)
topic/Thema (684)
url/URL (199)
finance/Finanzierung (151)
staff/Anzahl der Mitarbeiter (126)
legalform/Rechtsform (28)
link/Link (28)
verified/Verifiziert (26)
displayname/Anzeigename (24)
partei/Partei (20)
begin/Von (5)
end/Bis (4)
description/Beschreibung (3)
www/Webseite (2)


# Personen
__ Beispiel Person __

In [9]:
persons = Entities.find({'type' : 'person'})
persons.next()

{u'_id': ObjectId('54bd3c7a8b934da06340fe05'),
 u'aliases': [],
 u'created': datetime.datetime(2015, 1, 19, 17, 18, 50, 864000),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 1, 19, 17, 18, 50, 864000),
   u'desc': u'Quelle',
   u'format': u'link',
   u'id': u'7d5f63ae6645d9287f8a1c61579e136c1762f8c6feefdc6879d40789d1587104',
   u'key': u'source',
   u'updated': datetime.datetime(2015, 1, 19, 17, 18, 50, 864000),
   u'value': {u'remark': u'created by lobbyliste importer',
    u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}},
  {u'auto': True,
   u'created': datetime.datetime(2015, 1, 19, 17, 18, 50, 864000),
   u'desc': u'Titel',
   u'format': u'string',
   u'id': u'f50339f19881645fbf35aed74618e308b0c67be3461bb70a1c5cd58ffa0f44b3',
   u'key': u'titles',
   u'updated': datetime.datetime(2015, 1, 19, 17, 18, 50, 864000),
   u'value': u'Dr.med'}],
 u'importer': u'lobbyliste',
 u'name': u'Martin Warnach',
 u'search

__Datentypen für Personen__

- source/Quelle => dc:source
- titles/Titel => dc:title
- address/Adresse => vcard:ADR
- link/Link => vcard:RELATED
- surname/Nachname foaf:familyName
- names/Vornamen foaf:givenName
- photo/Foto => foaf:Image
* bundesland/Bundesland (629)
* btcertuid/Benutzename BT-Cert (608)
* wahlkreis/Wahlkreis (524)
* landesliste/Landesliste (194)
* partei/Partei (109)
- description/Beschreibung => dc:description
* url/URL (6)
- title/Titel => dc:title
* listenplatz/Listenplatz (1)
- www/Webseite => foaf:homepage

In [10]:
pipeline = [ {"$match" : { "type" : "person" }}, 
             {"$unwind": "$data"}, 
             {"$group": { "_id": { "$concat": [ "$data.key", "/", "$data.desc"]}, "count":{"$sum": 1}}}, 
             {"$sort": {"count":-1}} ]
for entity in Entities.aggregate(pipeline):
    print "%s (%i)" % (entity['_id'], entity['count'])

source/Quelle (20338)
titles/Titel (14560)
address/Adresse (6165)
link/Link (5399)
surname/Nachname (3444)
names/Vornamen (3442)
photo/Foto (2433)
bundesland/Bundesland (629)
btcertuid/Benutzename BT-Cert (608)
wahlkreis/Wahlkreis (524)
landesliste/Landesliste (194)
partei/Partei (109)
verified/Verifiziert (22)
displayname/Anzeigename (20)
description/Beschreibung (10)
url/URL (6)
title/Titel (5)
listenplatz/Listenplatz (1)
www/Webseite (1)


# Relationships

- Mitglied/member/mitglied => org:member
- executive => org:roleProperty => own:executiveOf (used with org:memberOf)

In [19]:
pipeline = [ {"$unwind": "$data"},
             {"$match" : {
                "data.key" : "position"
        }},
             {"$group": { "_id": "$data.value", "count":{"$sum": 1}}}, 
             {"$sort": {"count":-1}} ]
for entity in Relations.aggregate(pipeline):
    print "%s (%i)" % (entity['_id'], entity['count'])

Ordentliches Mitglied (993)
Stellvertretendes Mitglied (838)
Aufsichtsratsmitglied (460)
 (391)
Mitglied (165)
Vorstandsmitglied (161)
Staatssekretär (122)
Obmann (77)
Parlamentarischer Staatssekretär (74)
Obfrau (51)
Parlamentarische Staatssekretärin (48)
Stellvertretender Vorsitz (43)
Vorsitz (41)
Mitglied des Aufsichtsrats (40)
Vorstand (38)
Staatssekretärin (34)
Mitglied des Vorstands (33)
Präsidialmitglied (31)
Vorstandsvorsitz (31)
Vorstandsvorsitzender (30)
Aufsichtsratsvorsitz (29)
Persönliche Honorare für Vorträge (28)
Geschäftsführer (25)
Persönliche Honorare für Beratertätigkeit (24)
Mitglied des Kuratoriums (23)
Mitglied des Fernsehrats (23)
Präsident (22)
Mitglied im Verband (22)
Ministerpräsident (17)
Board of Directors (16)
Mitglied des Stiftungsrats (16)
Botschafter (15)
Berater (15)
Mitglied im Aufsichtsrat (15)
Landesminister (14)
Staatsminister (14)
Staatsrat (13)
Bundesminister (11)
Hauptgeschäftsführer (11)
Mitglied des Verwaltungsrats (11)
Beamteter Staatssekretär

In [104]:
pipeline = [ {"$group": { "_id": "$type", "count":{"$sum": 1}}}, 
             {"$sort": {"count":-1}} ]
for rel in Relations.aggregate(pipeline):
    print "%s (%i)" % (rel['_id'], rel['count'])

executive (17018)
member (4291)
donation (3820)
activity (2664)
business (1454)
position (1157)
government (580)
Mitglied (169)
association (64)
subsidiary (39)
Position (27)
committee (26)
Hausausweise (19)
mitglied (16)
consulting (5)
Tochterfirma (4)
sponsoring (3)
subisdiary (2)
ececutive (2)
Bundesdatenschutzbeauftragte (1)
Vorsitzender (1)
lobbyist (1)
publication (1)


In [31]:
relations = Relations.find({'type' : 'government'})

In [34]:
relations.next()

{u'_id': ObjectId('5501cb466e8aab706857b0c1'),
 u'created': datetime.datetime(2015, 3, 12, 17, 22, 14, 11000),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 3, 12, 17, 22, 14, 11000),
   u'desc': u'Position',
   u'format': u'string',
   u'id': u'fab2cb74049249ba75a1f0f7cb0243f03b057aca062352fd1e2d7a4d46c8e5be',
   u'key': u'position',
   u'updated': datetime.datetime(2015, 3, 12, 17, 22, 14, 11000),
   u'value': u'Staatsminister'},
  {u'auto': True,
   u'created': datetime.datetime(2015, 3, 12, 17, 22, 14, 11000),
   u'desc': u'Von',
   u'format': u'date',
   u'id': u'29d902a134d370c73438c68c72fe96222ee4bb6436239b18239245e1f4da6438',
   u'key': u'begin',
   u'updated': datetime.datetime(2015, 3, 12, 17, 22, 14, 11000),
   u'value': datetime.datetime(2013, 12, 16, 23, 0)}],
 u'entities': [ObjectId('54c71e92349d25992bca523c'),
  ObjectId('552ff9cdaf9ee96e1c1df7c5')],
 u'importer': u'kabinette',
 u'tags': [u'kabinette'],
 u'type': u'government',
 u'updated': datetime.d

- 

- source/Quelle => dc:source
- position/Position => skos:Role

In [101]:
pipeline = [ {"$unwind": "$data"}, 
             {"$group": { "_id": { "$concat": [ "$data.key", "/", "$data.desc"]}, "count":{"$sum": 1}}}, 
             {"$sort": {"count":-1}} ]
for rel in Relations.aggregate(pipeline):
    print "%s (%i)" % (rel['_id'], rel['count'])

source/Quelle (25018)
donation/Parteispende (9691)
position/Position (2735)
activity/Angaben zur Nebentätigkeit (2690)
position/Funktion im Ausschuss (2014)
begin/Von (622)
end/Ende (308)
end/Bis (247)
begin/Beginn (173)
start/Beginn (141)
issued/Ausgestellt von (18)
